In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from __init__ import *
import numpy as np
from export.saver import read_pickle, save_pickle
from read.svg_reader import SVGReader
from helpers.helpers import filter_none
from placement2.attract import *
from visuals.plotter import plot_general

In [3]:
case  = "amber_a_f01.svg"
sv = SVGReader(case)
sv.run()
# ag = AdjacencyGenerator(sv.layout, buffer_size=0.17)
# ag.run() # TODO shouldnt run if sv.layout is empty
# ag.draw_graph()
domains = sv.layout.domains


In [10]:

plot_general(sv.layout.domains)


In [53]:
from placement2.attract import collect_adjacent_nodes


sorted_doms = sorted(domains.values(), key=lambda n: (n.x.max))
nb_ranges = {d.name:collect_adjacent_nodes(d, domains) for d in sorted_doms}
nb_ranges


{'bedroom_1': {'transit_space': [3.22-3.36], 'laundry': [3.22-3.38]},
 'bedroom_2': {'kitchen': [3.22-3.36], 'transit_space': [3.22-3.36]},
 'laundry': {'bath': [4.98-5.30], 'm_bath': [4.98-5.30]},
 'kitchen': {'dining': [5.16-5.42]},
 'transit_space': {'bath': [5.16-5.30], 'dining': [5.16-5.42]},
 'm_bath': {'m_bedroom': [7.62-7.80]},
 'bath': {'m_bedroom': [7.62-7.80]},
 'dining': {'closet': [8.78-8.93], 'm_closet': [8.78-8.93]},
 'm_bedroom': None,
 'm_closet': None,
 'closet': None}